In [1]:
import pandas as pd
import time
from datetime import datetime
from pandas import ExcelWriter 
import matplotlib.pyplot as plt
from matplotlib import dates as mpl_dates
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
#import seaborn as sns
#sns.set()

pd.__version__




'1.2.3'

In [2]:
# temp=1588744463872/1000
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(temp))

NameError: name 'temp' is not defined

In [3]:
temp=1588744479826/1000
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(temp))

'2020-05-06 13:54:39'

In [4]:
    # Total number of seconds between dates

In [87]:
class JTLReader:
    def __init__(self,jtlPath):
        self.jtlPath=jtlPath
        self.jtlColumns = ['timeStamp', 'elapsed','label','threadName','dataType','bytes','sentBytes','URL','allThreads','success','responseCode','responseMessage']
        self.df_Jtl=pd.read_csv(self.jtlPath,usecols=self.jtlColumns,dtype={'label':'category'})#dtype={'label':'category'},nrows=195222    
        self.df_Jtl.sort_values(by=['timeStamp'],inplace=True) 
        self.df_Jtl.reset_index(drop=True, inplace=True)
        self.shape=self.df_Jtl.shape
        self.rows=self.shape[0]
        self.rows
        """Conver Epoc time to Actual Time Start"""
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Singapore')#Asia/Kolkata
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')
        """Conver Epoc time to Actual Time End"""
        
      

        """Saving master data frame to export to excel"""
        
        
        """Filter Transaction Controller & HTTP Request"""
        self.filt=(pd.isnull(self.df_Jtl['URL'])) #& (pd.isnull(self.df_Jtl['dataType']))
        self.df_jtl_TC=self.df_Jtl.loc[self.filt]
        self.df_jtl_HTTPReqeust=self.df_Jtl.loc[~self.filt]
        """Error Handling To be updated"""
        self.error_filt=(self.df_jtl_HTTPReqeust['success']==False) | (self.df_jtl_HTTPReqeust['success']==True)
        self.df_HttpErrors=self.df_jtl_HTTPReqeust.loc[self.error_filt]      
          
              
        
    """def getTransactionControllerData(self):
        print("Hello shiva")
        filt=(pd.isnull(self.df_Jtl['URL'])) & (pd.isnull(self.df_Jtl['dataType']))
        self.df_jtl_TC=self.df_Jtl.loc[filt]
        return self.df_jtl_TC"""
        

        
    def converTimeStamp(self):
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Singapore')#Africa/Cairo
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]') 
        
    def ms_to_sec(self):
         self.df_Jtl['elapsed']=((self.df_Jtl['elapsed'])/1000)
        
        
    def overAllResponseTimeSummary(self):
        self.df_overAllResponseTimeSummary=self.df_jtl_TC[['timeStamp','elapsed']]
        return self.df_overAllResponseTimeSummary
       
    def listOfErrors(self):
        return self.df_Jtl['responseCode'].value_counts()
    
    def shapeOfDaraFrame(self):
        return self.df_Jtl.shape
    
    def columnType(self,columnName):
        self.columnName=columnName
        return self.df_Jtl[self.columnName].dtype
    
    def getPeriod(self):
        self.startTime=self.df_Jtl['timeStamp'][0]
        self.endRow=self.df_Jtl['timeStamp'].shape[0]
        self.endTime=self.df_Jtl['timeStamp'][self.endRow-1]
        return f'{self.startTime} : {self.endTime}'
    
    def getDuration(self):
        self.startTime= self.df_Jtl['timeStamp'][0]
        self.endRow= self.df_Jtl['timeStamp'].shape[0]
        self.endTime= self.df_Jtl['timeStamp'][self.endRow-1]#-1
        self.executionStartTime=datetime(self.startTime.year,self.startTime.month,self.startTime.day,self.startTime.hour,self.startTime.minute,self.startTime.second)
        self.executionEndTime=datetime(self.endTime.year,self.endTime.month,self.endTime.day,self.endTime.hour,self.endTime.minute,self.endTime.second)
        self.timeDiff = self.executionEndTime-self.executionStartTime
        self.duration_in_s = self.timeDiff.total_seconds()
        self.duration=time.strftime("%H:%M:%S", time.gmtime(self.duration_in_s))
        return self.duration
    
    def getMaxRunningVusers(self):
        self.allThread=self.df_Jtl['allThreads'].max()
        return self.allThread 
    
      
    def getTotalHits(self):
        return self.df_jtl_HTTPReqeust.shape[0]
    
    def getAvgHitsPerSecond(self):
        return self.df_jtl_HTTPReqeust.shape[0]/self.duration_in_s
    
    def getTotalThroughput(self):
        self.totalThroughput=self.df_jtl_TC['bytes'].sum()
        return self.totalThroughput
    
    def getAvgThroughputPerSecond(self):
        self.avgThroughput=(self.df_jtl_TC['bytes'].sum())/self.duration_in_s
        return self.avgThroughput
    
    def getTotalPassFailFreqList(self):
        self.passFail_freq=self.df_jtl_TC['success'].value_counts().to_frame()
        #self.passFailFrequencyeqList=self.passFail_freq.index.tolist()
        
        return self.passFail_freq      
    
    def getTotalErrors(self):
        self.totalError=self.df_HttpErrors[['responseCode']].groupby(['responseCode'])
        self.totalError=self.df_HttpErrors['responseCode'].count()
        return self.totalError

    def getAnalysisSummary(self):
        self.period=self.getPeriod()
        self.duration=self.getDuration()
        self.maxRunningVusers=self.getMaxRunningVusers()
        self.totalThroughput=self.getTotalThroughput()
        self.avgThroughputPerSecond=self.getAvgThroughputPerSecond()
        self.totalHits=self.getTotalHits()
        self.avgHitsPerSecond=self.getAvgHitsPerSecond()
        self.totalErrors=self.getTotalErrors()
        self.df_AnalysisSummary=pd.DataFrame(
            {
                'Data':[
                        'Period','Duration','Maximum Running Vusers','Total bytes',
                        'Average bytes/second','TotalHits','Average Hits Per Second',
                        'Total Error','Total Pass','Total Failed'
                ],
                'Values':[
                            self.period,self.duration,self.maxRunningVusers,self.totalThroughput,
                            self.avgThroughputPerSecond,self.totalHits,self.avgHitsPerSecond,
                            self.totalErrors,self.getTotalPassFailFreqList(),'NA'
                ]
                
            }
        )
        return self.df_AnalysisSummary
    
    def responseTimeSummary(self):
        self.df_responseTimeSummary=self.df_jtl_TC[['label','elapsed']]
        self.df_txnGroup=self.df_responseTimeSummary.groupby(['label'],observed=True )  #observed=True      
        self.df_txnGroup=self.df_txnGroup['elapsed'].describe(percentiles=[.90])#self.df_txnGroup.describe(percentiles=[.90])
        self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(0)#self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(1)
        self.df_txnGroup = self.df_txnGroup[['count','min','mean','90%','max',]]
        self.df_txnGroup.rename(columns={'count':'#Samples','min':'Min','mean':'Average','90%':'90 Percentile','max':'Max'},inplace=True)
        ##self.df_txnGroup=self.df_txnErrorGroup.reset_index() """To be enable if you run describe () on data frame. Now we are running on specific column"""
        ##self.df_txnGroup['throughput']=self.df_txnGroup['count']/self.duration_in_s
        ##self.validTxn_filt=(self.df_txnGroup['count']>0)
        self.df_responseTimeSummaryOutput=self.df_txnGroup #self.df_txnGroup[self.validTxn_filt]
       
        return self.df_responseTimeSummaryOutput
    
    def getPassFailSummary(self):
        self.df_passFailSummary=self.df_jtl_TC[['label','success']]
        self.df_passFailGroup=self.df_passFailSummary.groupby(['label'],observed=True)#whe we use observed=True-->refer https://stackoverflow.com/questions/54922327/valueerror-operands-could-not-be-broadcast-together-with-shape-when-calling-pan?answertab=votes#tab-top
        self.df_txnGroup=self.df_passFailGroup['success'].value_counts()
        self.df_txnGroupSlice=self.df_txnGroup.loc[:,]
        self.df_passFailSummaryOutput=self.df_txnGroupSlice.unstack()
        self.df_passFailSummaryOutput=self.df_passFailSummaryOutput.fillna(0)
        self.df_passFailSummaryOutput.rename(columns={True:'Pass',False:'Fail'},inplace=True)
        return self.df_passFailSummaryOutput
    
    def getStartTimeOfEachGroup(self):
        self.transactionName=self.df_jtl_TC[['timeStamp','label']]
        self.transactionNameStartTimeByCatageory=self.transactionName.groupby('label', as_index = False,observed=True).nth(1)
        return self.transactionNameStartTimeByCatageory[['timeStamp','label']]
    
    def getEndTimeOfEachGroup(self):
        self.transactionName=self.df_jtl_TC[['timeStamp','label']]
        self.transactionNameEndTimeByCatageory=self.transactionName.groupby('label', as_index = False,observed=True).last()
        return self.transactionNameEndTimeByCatageory[['timeStamp','label']]
        
        
    
    def getReceiveSummary(self):
        print(self.duration_in_s)
        self.df_ReceiveSummary=self.df_jtl_TC[['label','bytes']]
        self.df_ReceiveSummary_grp=self.df_ReceiveSummary.groupby(['label'],observed=True)
        self.df_ReceiveSummary=self.df_ReceiveSummary.groupby('label', as_index = False,observed=True)['bytes'].sum()
        self.df_ReceiveSummary['bytes']=self.df_ReceiveSummary['bytes'].div(5107).round(2)
        self.df_ReceiveSummary['bytes']=self.df_ReceiveSummary['bytes'].div(1024).round(2)
        return  self.df_ReceiveSummary
        
        
    def getFinalResponseSummary(self):
        rs=pd.merge(self.df_responseTimeSummaryOutput,self.df_passFailSummaryOutput,left_index=True,right_index=True)
        if 'Fail' in rs.columns:# If transaction fail observed 
            rs['Error%']=((rs['Fail'])/(rs['#Samples']))*100 
            rs=rs[['#Samples','Pass','Fail','Error%','Min','Average','90 Percentile','Max']]#'Fail','Error%'
            return rs
        else:# All transactions pass
            rs=rs[['#Samples','Pass','Min','Average','90 Percentile','Max']]
            
            return rs
            
            
    
    def getRunnerVuserGraphData(self):
        '''T=min,S=Seconds'''       
        self.df_vusers=self.df_jtl_TC[['timeStamp','allThreads']]
        self.df_vusers.sort_values(by=['timeStamp'])
        self.df_vusers.set_index('timeStamp',inplace=True)
        self.df_vusers_summary = pd.DataFrame()
        self.df_vusers_summary['Vusers']=self.df_vusers['allThreads'].resample('10S').max()  
        self.df_vusers_summary.ffill(inplace=True)
        return self.df_vusers_summary
         
        
    
    def getHitsPersecondGraphData(self):
        '''T=min,S=Seconds'''
        #self.df_jtl_HTTPReqeust['hits']=1
        self.df_hits=self.df_jtl_HTTPReqeust[['timeStamp','URL']]#self.df_jtl_HTTPReqeust[['timeStamp','hits']]
        #self.df_hits.sort_values(by=['timeStamp'])
        self.df_hits.set_index('timeStamp',inplace=True)
        self.df_hits_summary = pd.DataFrame()
        self.df_hits_summary['hitsCount']=self.df_hits['URL'].resample('1T').count()/60   
        #df.loc[(df!=0).any(axis=1)]
        self.df_hits_summary=self.df_hits_summary.loc[(self.df_hits_summary!=0).any(axis=1)]
        return self.df_hits_summary
        #return self.df_hits
    
    def getNetworkReceivedPerSecondGraphData(self):
        '''T=min,S=Seconds'''
        self.df_byteReceived=self.df_jtl_TC[['timeStamp','bytes']]
        #self.df_byteReceived.sort_values(by=['timeStamp'])
        self.df_byteReceived.set_index('timeStamp',inplace=True)
        self.df_byteReceived_summary = pd.DataFrame()
        self.df_byteReceived_summary['Received']=(self.df_byteReceived['bytes']).resample('1T').sum()/60
        return self.df_byteReceived_summary.astype(int)
        
    
    def getThrouhputSendPersecondGraphData(self):
        '''T=min,S=Seconds'''
        self.df_throughputsend=self.df_jtl_TC[['timeStamp','sentBytes']]
        #self.df_throughputsend.sort_values(by=['timeStamp'])
        self.df_throughputsend.set_index('timeStamp',inplace=True)
        self.df_throughputsend_summary = pd.DataFrame()
        self.df_throughputsend_summary['ThroughputSend']=self.df_throughputsend['sentBytes'].resample('1T').sum()/60    
        return self.df_throughputsend_summary.astype(int)
    
    def getErrorPersecondGraphData(self):
        '''T=min,S=Seconds'''
        self.df_errorCode=self.df_HttpErrors[['timeStamp','responseCode']]
        #self.df_throughputsend.sort_values(by=['timeStamp'])
        self.df_errorCode.set_index('timeStamp',inplace=True)
        self.df_errorCode_summary = pd.DataFrame()
        self.df_errorCode_summary['HTTPError']=self.df_errorCode['responseCode'].resample('1T').count() 
        return self.df_errorCode_summary
    
    def gerErrorPivot(self):
        self.df_df_HttpErrors_pivot=self.df_HttpErrors[['timeStamp','responseCode','responseMessage']]
        return self.df_df_HttpErrors_pivot.pivot_table(index=pd.Grouper(freq='T',key='timeStamp'),columns="responseMessage",aggfunc="count")        
 
    
    def getOverallResponseTime(self):
        return self.df_jtl_TC[['timeStamp','elapsed','label']]
    
    def getRawDataFrame(self):
        return self.df_Jtl
    
#     def get errorSummary(self):
#         self.filt_error=(self.df_jtl_HTTPReqeust['responseCode']!=200) & (self.df_jtl_HTTPReqeust['responseCode']!=202)
#         self.df_error=self.df_jtl_HTTPReqeust[self.filt_error]
#         return self.df_error
    
    def temp(self):
        self.slicer_grp=self.df_jtl_TC[['timeStamp','elapsed','label']]
        self.slicer_grp.sort_values(by=['timeStamp'])
        self.slicer_grp.set_index('timeStamp',inplace=True)
        self.df_response_summary_slicer = pd.DataFrame()
        self.df_slicer_txnGroup=self.slicer_grp.groupby(['label'],observed=True )
        self.df_response_summary_slicer=self.df_slicer_txnGroup['elapsed'].resample('30T').agg(['min','mean','max','count'])
        return self.df_response_summary_slicer
    
    def getDetailHttpErrors(self):
        self.df_errors=self.df_HttpErrors[['timeStamp','label','responseCode','responseMessage']]
        return  self.df_errors
    
    def getHttpErrorSummary(self):
        self.df_HttpErrorSummary=self.df_HttpErrors[['responseCode','responseMessage']].groupby(['responseCode'])
        self.df_HttpErrorSummary=self.df_HttpErrors['responseCode'].value_counts()
        return self.df_HttpErrorSummary
    
    
      
obj1=JTLReader("D:\\Projects\\JMR\\results\\LT_75Users.jtl") 
#obj1.converTimeStamp()
obj1.ms_to_sec() 
obj1.getDuration()



'01:15:09'

In [91]:
temp=obj1.getStartTimeOfEachGroup()
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 3 to 745
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timeStamp  7 non-null      datetime64[ns]
 1   label      7 non-null      category      
dtypes: category(1), datetime64[ns](1)
memory usage: 795.0 bytes


In [89]:
temp1=obj1.getEndTimeOfEachGroup()
temp1

timeStamp                           label
0 2020-12-28 16:22:56                  TC01_Txn01_URL
1 2020-12-28 16:23:07                TC01_Txn02_Login
2 2020-12-28 16:23:19           TC01_Txn03_UploadFile
3 2020-12-28 17:09:27    TC01_Txn04_01_Validate_50Rec
4 2020-12-28 17:14:17  TC01_Txn04_02_Validate_1000Rec
5 2020-12-28 17:19:45    TC01_Txn04_03_Validate_5KRec
6 2020-12-28 17:23:06   TC01_Txn04_04_Validate_10KRec

In [86]:
#obj1.gerErrorPivot()

In [44]:
#obj1.getHttpErrorSummary()

In [45]:
#y=obj1.getTotalPassFailFreqList()
#y

In [46]:
#x=obj1.temp()
#x

In [47]:
#obj1.df_Jtl

In [48]:
#obj1.df_jtl_HTTPReqeust

In [49]:
#obj1.df_jtl_TC

In [50]:
#obj1.getPeriod()

In [51]:
#obj1.getDuration()

In [52]:
#obj1.getMaxRunningVusers()

In [53]:
#obj1.getTotalThroughput()

In [54]:
"""Analysis Summery"""
analysisSummary=obj1.getAnalysisSummary()
analysisSummary


Data                                        Values
0                   Period     2020-12-28 16:07:57 : 2020-12-28 17:23:06
1                 Duration                                      01:15:09
2   Maximum Running Vusers                                            29
3              Total bytes                                     401680737
4     Average bytes/second                                  89084.217565
5                TotalHits                                          1083
6  Average Hits Per Second                                      0.240186
7              Total Error                                          1083
8               Total Pass         success
False      568
True       440
9             Total Failed                                            NA

In [ ]:
"""responseTimeSummary"""
pd.set_option('display.max_rows',200)
responseTimeSummary=obj1.responseTimeSummary()
responseTimeSummary.head(100)

In [ ]:
"""Get Pass and Fail Summary"""
passandfail=obj1.getPassFailSummary()
passandfail

In [ ]:
"""Get Final Response Summary"""
rs=obj1.getFinalResponseSummary()
rs

In [ ]:
"""Get Vuser Summary"""
runningVuser=obj1.getRunnerVuserGraphData()
runningVuser

In [ ]:
"""Get Hits Summary"""
hitsPersecond=obj1.getHitsPersecondGraphData() 
hitsPersecond

In [ ]:
"""Get Throughput Received Summery"""
Received=obj1.getNetworkReceivedPerSecondGraphData()
Received

In [ ]:
"""Get Throughput Send Summery"""
throughputsend=obj1.getThrouhputSendPersecondGraphData()
throughputsend

In [ ]:
"""Get Error Summary"""
errors=obj1.getErrorPersecondGraphData()
errors

In [ ]:
"""overAllResponseTimeSummary"""
overallResponseTime=obj1.getOverallResponseTime()
overallResponseTime

In [ ]:
detailerrors=obj1.getDetailHttpErrors()
detailerrors

In [ ]:
errorPivot=obj1.gerErrorPivot()

In [ ]:
obj1.getHttpErrorSummary()

In [ ]:
rawdataframe=obj1.getRawDataFrame()
rawdataframe

In [ ]:
"""Generate Excel Report"""
writer = ExcelWriter('PerformanceTestReport.xlsx',mode='w')
#Write Analysis Summary to Excel
analysisSummary.to_excel(writer,sheet_name='AnalysisSummary')
#write response summary to excel with txn name,min,avg,max,90% & max
rs.to_excel(writer,sheet_name='ResponseSummary')
#Write Running Vusers
runningVuser.to_excel(writer,sheet_name='RunningVuserSummary')
#Write Hits Summary
hitsPersecond.to_excel(writer,sheet_name='HitsSummary')
#Write Throughput Received Summary
Received.to_excel(writer,sheet_name='ReceivedSummary')
#Write Throughput Send Summary
throughputsend.to_excel(writer,sheet_name='ThroughputSendSummary')
#Write error summary
errors.to_excel(writer,sheet_name='ErrorGraph')
#Write ResponseTime
overallResponseTime.to_excel(writer,sheet_name='ResponseTimeGraph')
#Write Detail Errors
#detailerrors.to_excel(writer,sheet_name='DetailErrors')
#Write Error Pivot
errorPivot.to_excel(writer,sheet_name='ErrorPivot')
#Export Original dataframe
#rawdataframe.to_excel(writer,sheet_name='dataframe')
writer.save()







#Draw Running Vuser Graph

In [ ]:
df_vuser=obj1.getRunnerVuserGraphData()
df_vuser.reset_index('timeStamp',inplace=True)
#Draw Graph
plt.style.use('fivethirtyeight')#print(plt.style.available)
plt.figure(figsize=(25,7))
plt.plot(df_vuser['timeStamp'],df_vuser['Vusers'],label='Vusers')#color="m",linestyle='-',linewidth=3,marker='.' #Important Line x & Y axis #crimson
plt.title("Running Vusers",fontsize=14,fontweight="bold")
plt.xlabel("Time",fontsize=13)#,fontweight="bold"
plt.ylabel("Number of vuser",fontsize=13)#,fontweight="bold"
plt.legend(loc='upper left')#This will take label name from plt.plot. So no need to pass the labe in sequence order
plt.gcf().autofmt_xdate()#plt.xticks(rotation=45,fontsize=13)
date_format=mpl_dates.DateFormatter('%b, %d %Y')#To formate the x axis with date format
plt.gca().xaxis.set_major_formatter(date_format)#To formate the x axis with date format
plt.tight_layout()#Automatically render without padding issue #plt.grid(True)
plt.show()
#plt.savefig("Vuser.png")


In [ ]:
#Draw Hits Summary

In [ ]:
df_hits=obj1.getHitsPersecondGraphData()
df_hits.reset_index('timeStamp',inplace=True)
#Draw Graph
plt.style.use('fivethirtyeight')#print(plt.style.available)
plt.figure(figsize=(25,7))
plt.plot(df_hits['timeStamp'],df_hits['hitsCount'],label='Hits')#color="m",linestyle='-',linewidth=3,marker='.' #Important Line x & Y axis #crimson
plt.title("Hits",fontsize=14,fontweight="bold")
plt.xlabel("Time",fontsize=13)#,fontweight="bold"
plt.ylabel("Hits",fontsize=13)#,fontweight="bold"
plt.legend(loc='upper left')#This will take label name from plt.plot. So no need to pass the labe in sequence order
plt.gcf().autofmt_xdate()#plt.xticks(rotation=45,fontsize=13)
date_format=mpl_dates.DateFormatter('%b, %d %Y')#To formate the x axis with date format
plt.gca().xaxis.set_major_formatter(date_format)#To formate the x axis with date format
plt.tight_layout()#Automatically render without padding issue #plt.grid(True)
plt.show()
#plt.savefig("Vuser.png")

In [ ]:
#Draw Throughput Received

In [ ]:
df_throughput=obj1.getThrouhputPersecondGraphData()
df_throughput.reset_index('timeStamp',inplace=True)


#Draw Graph
plt.style.use('fivethirtyeight')#print(plt.style.available)
plt.figure(figsize=(25,7))
plt.plot(df_throughput['timeStamp'],df_throughput['ThroughputReceived'],label='Throughput Received')#color="m",linestyle='-',linewidth=3,marker='.' #Important Line x & Y axis #crimson
plt.title("Throughput Received",fontsize=14,fontweight="bold")
plt.xlabel("Time",fontsize=13,fontweight="bold")#,fontweight="bold"
plt.ylabel("Bytes",fontsize=13,fontweight="bold")#,fontweight="bold"
plt.legend(loc='upper left')#This will take label name from plt.plot. So no need to pass the labe in sequence order
plt.gcf().autofmt_xdate()#plt.xticks(rotation=45,fontsize=13)
ax = plt.gca()
ax.ticklabel_format(useOffset=False, style='plain', axis='y')#To remove scientifice numbers
date_format=mpl_dates.DateFormatter('%b, %d %Y')#To formate the x axis with date format
plt.gca().xaxis.set_major_formatter(date_format)#To formate the x axis with date format
plt.tight_layout()#Automatically render without padding issue #plt.grid(True)
plt.show()


In [ ]:
#Draw Throughput Send

In [ ]:
df_throughput_send=obj1.getThrouhputSendPersecondGraphData()
df_throughput_send.reset_index('timeStamp',inplace=True)

#Draw Graph
plt.style.use('fivethirtyeight')#print(plt.style.available)
plt.figure(figsize=(25,7))
plt.plot(df_throughput_send['timeStamp'],df_throughput_send['ThroughputSend'],label='Throughput Send')#color="m",linestyle='-',linewidth=3,marker='.' #Important Line x & Y axis #crimson
plt.title("Throughput Send",fontsize=14,fontweight="bold")
plt.xlabel("Time",fontsize=13,fontweight="bold")#,fontweight="bold"
plt.ylabel("Bytes",fontsize=13,fontweight="bold")#,fontweight="bold"
plt.legend(loc='upper left')#This will take label name from plt.plot. So no need to pass the labe in sequence order
plt.gcf().autofmt_xdate()#plt.xticks(rotation=45,fontsize=13)
ax = plt.gca()
ax.ticklabel_format(useOffset=False, style='plain', axis='y')#To remove scientifice numbers
date_format=mpl_dates.DateFormatter('%b, %d %Y')#To formate the x axis with date format
plt.gca().xaxis.set_major_formatter(date_format)#To formate the x axis with date format
plt.tight_layout()#Automatically render without padding issue #plt.grid(True)
plt.show()


In [ ]:
#Draw Error Grph

In [ ]:
df_errors=obj1.getErrorPersecondGraphData()
df_errors.reset_index('timeStamp',inplace=True)

#Draw Graph
plt.style.use('fivethirtyeight')#print(plt.style.available)
plt.figure(figsize=(25,7))
plt.plot(df_errors['timeStamp'],df_errors['HTTPError'],label='Error Summary')#color="m",linestyle='-',linewidth=3,marker='.' #Important Line x & Y axis #crimson
plt.title("Error Summary",fontsize=14,fontweight="bold")
plt.xlabel("Time",fontsize=13,fontweight="bold")#,fontweight="bold"
plt.ylabel("Number of Errors",fontsize=13,fontweight="bold")#,fontweight="bold"
plt.legend(loc='upper left')#This will take label name from plt.plot. So no need to pass the labe in sequence order
plt.gcf().autofmt_xdate()#plt.xticks(rotation=45,fontsize=13)
ax = plt.gca()
ax.ticklabel_format(useOffset=False, style='plain', axis='y')#To remove scientifice numbers
date_format=mpl_dates.DateFormatter('%b, %d %Y')#To formate the x axis with date format
plt.gca().xaxis.set_major_formatter(date_format)#To formate the x axis with date format
plt.tight_layout()#Automatically render without padding issue #plt.grid(True)
plt.show()

In [ ]:
#Draw Vuser Plotly

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
import random
#'rgb(131, 90, 241)','rgb(127, 166, 238)','rgb(111, 231, 219)','rgb(184, 247, 212)'
colors = ['rgb(128,128,0)']

df_vuser=obj1.getRunnerVuserGraphData()
df_vuser.reset_index('timeStamp',inplace=True)

x_values=df_vuser['timeStamp']
y_values=df_vuser['Vusers']


trace1=go.Scatter(x=x_values,y=y_values,mode='lines',showlegend=True,stackgroup='one',name='runningvusers',line=dict(width=2, color=random.choice(colors)),connectgaps=True)#stackgroup='one',fillcolor='rgba(231,107,243,0.2)'
data=[trace1]
layout=go.Layout(title='Running Vuser Graph',xaxis=dict(title='Time'),yaxis=dict(title='Vusers'),hovermode='closest')

fig=go.Figure(data=data,layout=layout)
fig.update_layout(
    template = 'plotly_dark',
    autosize=False,
    width=1200,
    height=400,
    plot_bgcolor='rgba(0,0,0,0)',    
    legend=dict(x=0.1,y=0.8,traceorder='normal'),
    title=dict(text='Running Vuser Graph',y=0.9,x=0.5,xanchor='center',yanchor='top'),
    title_font_family="Times New Roman",
    title_font_color='rgb(255,255,255)',

  annotations=[
        dict(xref='paper',x=0.90,y=40,xanchor='auto', yanchor='auto',font=dict(family='Arial',size=10),showarrow=False,
            text='Displays the number of Vusers that executed Vuser scripts,and their status,\nduring each second of a load test.\nThis graph is useful for determining the Vuser load on your server at any given moment')
    ]

)

#fig.update_xaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')
#fig.update_yaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')


pyo.plot(fig,filename='RunningVusers.html')

In [ ]:
#Draw Hits Plotly

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
import random
#'rgb(131, 90, 241)','rgb(127, 166, 238)','rgb(111, 231, 219)','rgb(184, 247, 212)'
colors = ['rgb(127, 166, 238)']

df_hits=obj1.getHitsPersecondGraphData()
df_hits.reset_index('timeStamp',inplace=True)

x_values=df_hits['timeStamp']
y_values=df_hits['hitsCount']


trace1=go.Scatter(x=x_values,y=y_values,mode='lines',showlegend=True,name='Hits',line=dict(width=2, color=random.choice(colors)),connectgaps=True)#stackgroup='one',fillcolor='rgba(231,107,243,0.2)'
data=[trace1]
layout=go.Layout(title='Hits Graph',xaxis=dict(title='Time'),yaxis=dict(title='Hits'),hovermode='closest')

fig=go.Figure(data=data,layout=layout)
fig.update_layout(
    template = 'plotly_dark',
    autosize=False,
    width=1200,
    height=400,
    plot_bgcolor='rgba(0,0,0,0)',    
    legend=dict(x=0.8,y=1,traceorder='normal'),
    title=dict(text='Hits Graph',y=0.9,x=0.5,xanchor='center',yanchor='top'),
    title_font_family="Times New Roman",
    title_font_color='rgb(255,255,255)',

  annotations=[
        dict(xref='paper',x=0.90,y=40,xanchor='auto', yanchor='auto',font=dict(family='Arial',size=10),showarrow=False,
            text='Displays the number of hits made on the Web server by Vusers during each second of the load test.This graph helps you evaluate the amount of load Vusers generate, in terms of the number of hits')
    ]

)

#fig.update_xaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')
#fig.update_yaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')


pyo.plot(fig,filename='HitsGraph.html')

In [ ]:
#Draw Throughput Receive Graph

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
import random
#'rgb(131, 90, 241)','rgb(127, 166, 238)','rgb(111, 231, 219)','rgb(184, 247, 212)'
colors = ['rgb(111, 231, 219)']

df_throughput=obj1.getThrouhputPersecondGraphData()
df_throughput.reset_index('timeStamp',inplace=True)

x_values=df_throughput['timeStamp']
y_values=df_throughput['ThroughputReceived']


trace1=go.Scatter(x=x_values,y=y_values,mode='lines',showlegend=True,stackgroup='one',name='Throughput Received',line=dict(width=2, color=random.choice(colors)),connectgaps=True)#stackgroup='one',fillcolor='rgba(231,107,243,0.2)'
data=[trace1]
layout=go.Layout(title='Throughput Receive Graph',xaxis=dict(title='Time'),yaxis=dict(title='bytes'),hovermode='closest')

fig=go.Figure(data=data,layout=layout)
fig.update_layout(
    template = 'plotly_dark',
    autosize=False,
    width=1200,
    height=400,
    plot_bgcolor='rgba(0,0,0,0)',    
    legend=dict(x=0.8,y=1,traceorder='normal'),
    title=dict(text='Throughput Receive Graph',y=0.9,x=0.5,xanchor='center',yanchor='top'),
    title_font_family="Times New Roman",
    title_font_color='rgb(255,255,255)',

  annotations=[  #['auto', 'left', 'center', 'right']        #['auto', 'top', 'middle', 'bottom']
        dict(xref='paper',x=0.90,y=90,xanchor='right', yanchor='top',font=dict(family='Arial',size=10),showarrow=False,
            text='Displays the amount of throughput (in bytes) on the Web server during the load test.Throughput represents the amount of data that the Vusers received from the server ')
    ]

)

#fig.update_xaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')
#fig.update_yaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')


pyo.plot(fig,filename='ThroughputReceiveGraph.html')

In [ ]:
#Draw Throughput Send Graph

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
import random
#'rgb(131, 90, 241)','rgb(127, 166, 238)','rgb(111, 231, 219)','rgb(184, 247, 212)'
colors = ['rgb(131, 90, 241)']

df_throughput_send=obj1.getThrouhputSendPersecondGraphData()
df_throughput_send.reset_index('timeStamp',inplace=True)

x_values=df_throughput_send['timeStamp']
y_values=df_throughput_send['ThroughputSend']


trace1=go.Scatter(x=x_values,y=y_values,mode='lines',showlegend=True,stackgroup='one',name='Throughput Send',line=dict(width=2, color=random.choice(colors)),connectgaps=True)#stackgroup='one',fillcolor='rgba(231,107,243,0.2)'
data=[trace1]
layout=go.Layout(title='Throughput Send Graph',xaxis=dict(title='Time'),yaxis=dict(title='bytes'),hovermode='closest')

fig=go.Figure(data=data,layout=layout)
fig.update_layout(
    template = 'plotly_dark',
    autosize=False,
    width=1200,
    height=400,
    plot_bgcolor='rgba(0,0,0,0)',    
    legend=dict(x=0.8,y=1,traceorder='normal'),
    title=dict(text='Throughput Send Graph',y=0.9,x=0.5,xanchor='center',yanchor='top'),
    title_font_family="Times New Roman",
    title_font_color='rgb(255,255,255)',

  annotations=[  #['auto', 'left', 'center', 'right']        #['auto', 'top', 'middle', 'bottom']
        dict(xref='paper',x=1,y=90,xanchor='right', yanchor='top',font=dict(family='Arial',size=10),showarrow=False,
            text='Displays the amount of throughput (in bytes) on the Web server during the load test.Throughput represents the amount of data that the Vusers send to the server at any given second.This graph helps you to evaluate the amount of load ')
    ]

)

#fig.update_xaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')
#fig.update_yaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')


pyo.plot(fig,filename='ThroughputSendGraph.html')

In [ ]:
#Draw Error Graph

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
import random
#'rgb(131, 90, 241)','rgb(127, 166, 238)','rgb(111, 231, 219)','rgb(184, 247, 212)'
colors = ['rgb(127, 166, 238)']

df_errors=obj1.getErrorPersecondGraphData()
df_errors.reset_index('timeStamp',inplace=True)

x_values=df_errors['timeStamp']
y_values=df_errors['HTTPError']


trace1=go.Scatter(x=x_values,y=y_values,mode='lines',showlegend=True,stackgroup='one',name='Errors',line=dict(width=2, color=random.choice(colors)),connectgaps=True)#stackgroup='one',fillcolor='rgba(231,107,243,0.2)'
data=[trace1]
layout=go.Layout(title='Error Graph',xaxis=dict(title='Time'),yaxis=dict(title='Number of Errors'),hovermode='closest')

fig=go.Figure(data=data,layout=layout)
fig.update_layout(
    template = 'plotly_dark',
    autosize=False,
    width=1200,
    height=400,
    plot_bgcolor='rgba(0,0,0,0)',    
    legend=dict(x=0.8,y=1,traceorder='normal'),
    title=dict(text='Errors Graph',y=0.9,x=0.5,xanchor='center',yanchor='top'),
    title_font_family="Times New Roman",
    title_font_color='rgb(255,255,255)',

 
)

#fig.update_xaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')
#fig.update_yaxes(showgrid=True, gridwidth=0.1, gridcolor='rgb(105,105,105)')


pyo.plot(fig,filename='Errors.html')